In [36]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234) 
}

In [37]:
# datasets = ['breast', 'campus', 'churn', 'climate',
#             'compas', 'diabetes', 'german', 'heart',
#             'stroke', 'student', 'water', 'adult']

datasets = ["credit"]

In [38]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
n_datasets = len(datasets)
n_splits = 10
# repeats 5, splits 2
skf = StratifiedKFold(n_splits=n_splits, random_state=1234, shuffle=True)

auc_scores = np.zeros((len(clfs)+1, n_datasets, n_splits))
loss = np.zeros((len(clfs)+1, n_datasets, n_splits))

In [39]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(skf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])

            # if(clf_name == "GNB" and dataset=="compas"):
            #     clf.fit(X.iloc[train], y.iloc[train])
            #     y_preds = clf.predict(X.iloc[test])
            #     fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            #     auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            # else:
            clf_pipeline = make_pipeline(
                preprocess,
                clf
            )
                
            clf_pipeline.fit(X.iloc[train], y.iloc[train])
            y_preds = clf_pipeline.predict(X.iloc[test])
            fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
            auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            loss[clf_id, data_id, fold_id] = log_loss(y.iloc[test], y_preds)

            

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (s

In [40]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(skf.split(X, y)):

        clf_pipeline = make_pipeline(
                preprocess,
                DecisionListClassifier(random_state=1234)
            )
                
        clf_pipeline.fit(X.iloc[train], y.iloc[train])
        y_preds = clf_pipeline.predict(X.iloc[test])
        fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
        auc_scores[5, data_id, fold_id] = metrics.auc(fpr, tpr)
        loss[5, data_id, fold_id] = log_loss(y.iloc[test], y_preds)
        

In [41]:
auc_scores

array([[[0.75      , 0.83888889, 0.89444444, 0.7547619 , 0.89047619,
         0.87619048, 0.71264368, 0.8204023 , 0.81321839, 0.69588123]],

       [[0.90555556, 0.96944444, 0.925     , 0.89285714, 0.85952381,
         0.86190476, 0.7710728 , 0.85823755, 0.88936782, 0.87547893]],

       [[0.89722222, 0.94166667, 0.90833333, 0.86428571, 0.85952381,
         0.87857143, 0.7710728 , 0.84770115, 0.84770115, 0.89272031]],

       [[0.63611111, 0.66944444, 0.65833333, 0.61904762, 0.7547619 ,
         0.78571429, 0.79645594, 0.84099617, 0.59291188, 0.63793103]],

       [[0.89444444, 0.95833333, 0.90833333, 0.84761905, 0.8452381 ,
         0.8452381 , 0.79885057, 0.83045977, 0.86494253, 0.87547893]],

       [[0.9       , 0.95833333, 0.89722222, 0.86428571, 0.86666667,
         0.88095238, 0.74664751, 0.84434866, 0.85775862, 0.8855364 ]]])

In [42]:
auc_scores[0,:, 0]

array([0.75])

In [43]:
auc_scores[1,:, 0]

array([0.90555556])

In [44]:
auc_scores[2,:, 0]

array([0.89722222])

In [45]:
auc_scores[3,:, 0]

array([0.63611111])

In [46]:
auc_scores[4,:, 0]

array([0.89444444])

In [47]:
np.save('./test_results/auc/auc_results', auc_scores)
np.save('./test_results/auc/auc_losses', loss)